In [7]:
!pip install segmentation_models
!pip install albumentations==0.4.5

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import tensorflow as tf

from IPython.display import clear_output
import matplotlib.pyplot as plt

In [10]:
from PIL import Image

import os
import glob
import numpy as np

from sklearn import model_selection

import segmentation_models as sm

In [11]:
from src.DataLoader import DataLoader, make_augmentation
from src.Model import unet_model

In [16]:
base_dir = "/storage/supervisely/processed"
ims_dir = os.path.join(base_dir, "imgs")
labels_dir = os.path.join(base_dir, "labels")
img_files = sorted(glob.glob(ims_dir + "/*.*"))
mask_files = sorted(glob.glob(labels_dir + "/*.png"))
print (f"Masks count : {len(mask_files)};\nImgs count : {len(img_files)};")

Masks count : 5709;
Imgs count : 5709;


In [17]:
BATCH_SIZE = 24
dataset = list(zip(img_files, mask_files))
train_dataset, test_dataset = model_selection.train_test_split(dataset, test_size=0.2, random_state=0)
print(f"Train dataset size {len(train_dataset)}; Test dataset size {len(test_dataset)}")

Train dataset size 4567; Test dataset size 1142


In [18]:
output_size = (256, 256)

training_augmentation = make_augmentation(output_size=output_size, is_validation=False)
train_data_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, augmentation_fn=training_augmentation, output_size=output_size, shuffle=True)

validation_augmentation = make_augmentation(output_size=output_size, is_validation=True)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, augmentation_fn=validation_augmentation, output_size=output_size, shuffle=False)

### Create model

In [25]:
OUTPUT_CHANNELS = 1
model = unet_model(OUTPUT_CHANNELS)

In [26]:
model.load_weights("/storage/training_result/models/best_model_full_unwrapped_from_scratch_lr00001_batch24.h5")

### Test predictions on a single batch

In [29]:
batch = test_data_loader[0]
image_batch = batch[0]
mask_batch = batch[1]
print (f"Image batch size {len(image_batch)}") 

Image batch size 24


In [ ]:
res = model.predict(image_batch, batch_size=len(image_batch))
res.shape

In [ ]:
def denormalize(img):
  mean=(0.485, 0.456, 0.406)
  std=(0.229, 0.224, 0.225)
  # img = test_data_loader[batch_idx][0][img_idx]
  reverse = ((img * std + mean) * 255).astype('uint8')
  return reverse

In [ ]:
for i in range(16):
  fig, axes = plt.subplots(1, 3, figsize=(8, 8))
  axes[0].imshow(res[i].squeeze())
  axes[1].imshow(denormalize(image_batch[i]))
  axes[2].imshow(mask_batch[i].squeeze())

In [ ]:
import pandas as pd
import tqdm

## Get predictions for all test dataset

In [ ]:
res = model.predict(test_data_loader,  steps=len(test_data_loader), verbose=1)

Estimate losses on every image

In [ ]:
dice_loss = sm.losses.DiceLoss()
binary_focal_loss = sm.losses.BinaryFocalLoss()
total_loss = sm.losses.DiceLoss() + (1 * sm.losses.BinaryFocalLoss())
losses = {
          "dice": dice_loss,
          "bin focal" : binary_focal_loss,
          "total" : total_loss 
}

"""
res_copy =  res[0].squeeze().copy()
thr = 0.2
res_copy[res_copy <= thr] = 0.0
res_copy[res_copy > thr] = 1.0
plt.figure(figsize=(10, 10))
plt.imshow(res_copy)
"""

all_losses = []

batch_size = 16

for idx in tqdm.tqdm(range(len(res))):
  MASKS_IDX = 1
  batch_idx = idx // batch_size
  img_idx = idx - batch_idx * batch_size

  gt = test_data_loader[batch_idx][MASKS_IDX][img_idx]  
  pred = res[idx]

  img_losses = []
  for loss_name, loss_func in losses.items():
    loss_res = float(loss_func(gt, pred))
    # loss_res = val(test_data_loader[0][1][0], np.expand_dims(res_copy, -1))
    # print("{} : {}".format(key, loss_res))
    img_losses.append(loss_res)
  all_losses.append(img_losses)


In [ ]:
losses_data = pd.DataFrame(all_losses, columns = ["dice", "focal", "total"])
losses_data

In [ ]:
losses_data.hist(bins=100, figsize=(10, 10));

In [ ]:
losses_data.describe()

In [ ]:
def denormalize(img):
  mean=(0.485, 0.456, 0.406)
  std=(0.229, 0.224, 0.225)
  # img = test_data_loader[batch_idx][0][img_idx]
  reverse = ((img * std + mean) * 255).astype('uint8')
  return reverse

### Get 10 images from the best to the worst, sorted by losses

In [ ]:
sorted_losses = losses_data.sort_values(by="total")

In [ ]:
names = []
for img_idx in np.linspace(start=0, stop=len(sorted_losses), num=10, endpoint=False):
  item =sorted_losses.iloc[int(img_idx)]
  print(item.name, item.total)
  names.append(sorted_losses.iloc[int(img_idx)].name)

and show them

In [ ]:
for idx in names:
  fig, axes = plt.subplots(1, 3)

  batch_idx = idx // batch_size
  img_idx = idx - batch_idx * batch_size
  
  reverse = denormalize(test_data_loader[batch_idx][0][img_idx])
  axes[0].imshow(reverse.squeeze())
  axes[1].imshow(test_data_loader[batch_idx][1][img_idx].squeeze())
  axes[2].imshow(res[idx].squeeze())  

In [ ]:
fig, axes = plt.subplots(1, 3)

idx = sorted_losses.iloc[-1].name
batch_idx = idx // batch_size
img_idx = idx - batch_idx * batch_size


axes[0].imshow(res[idx].squeeze())  
# axes[1].imshow(test_data_loader[-1][1][-2].squeeze())
axes[1].imshow(test_data_loader[batch_idx][1][img_idx].squeeze())
# reverse = denormalize(test_data_loader[-1][0][-2])
reverse = denormalize(test_data_loader[batch_idx][0][img_idx])
axes[2].imshow(reverse.squeeze())

In [ ]:
plt.imshow(image_batch[0])